In [1]:
import os, sys
import pandas as pd
import numpy as np
import datetime as dt
from pprint import pprint
import csv
import itertools as it
from prettytable import PrettyTable
# from nanrms import *
# from handover import *
# from mask import *
import matplotlib.pylab as plt
import seaborn as sns

pdir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # for jupyter-notebook
sys.path.insert(1, pdir)
from myutils import *

%config InlineBackend.figure_format = 'retina'  # 提高 jupyter notebook 的圖形顯示解析度
# plt.style.use('ggplot')

pd.set_option('display.max_columns', 200)
# pd.set_option('display.max_rows', 200)

def set_data(df):
    df['seq'] = df['seq'].astype('Int32')
    # df['rpkg'] = df['rpkg'].astype('Int8')
    df['frame_id'] = df['frame_id'].astype('Int32')
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['xmit_time'] = pd.to_datetime(df['xmit_time'])
    df['arr_time'] = pd.to_datetime(df['arr_time'])
    df['Timestamp_epoch'] = df['Timestamp_epoch'].astype('float32')
    df['xmit_time_epoch'] = df['xmit_time_epoch'].astype('float32')
    df['arr_time_epoch'] = df['arr_time_epoch'].astype('float32')
    df['lost'] = df['lost'].astype('boolean')
    df['excl'] = df['excl'].astype('boolean')
    df['latency'] = df['latency'].astype('float32')
    return df

datadir = "/Users/jackbedford/Desktop/MOXA/Code/data"
path = "./temp2"
exps = {
    "2023-05-04/_Bandlock_8_Schemes_Phone": ["#{:02d}".format(i+1) for i in range(6)],
}
title = '8_Schemes'
_devices = [
    ["sm00", "sm01", "sm02", "sm03", "sm04", "sm05", "sm06", "sm07",],
]
_schemes = [
    ["All", "B3", "B7", "B8", "B3B7", "B3B8", "B7B8", "LTE",],
]

In [3]:
i = -1
dfs_ul = []
for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_ul.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_uplk_loss_latency.csv')
            data = os.path.join(datadir, exp, dev, trace, 'data', 'udp_uplk_loss_latency.csv')
            print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df = set_data(df)
            dfs_ul[i].append(df.copy())
            st.append(df['seq'].array[0])
            et.append(df['seq'].array[-1])
        st, et = max(st), min(et)
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            dfs_ul[i][j] = dfs_ul[i][j][(dfs_ul[i][j]['seq'] >= st) & (dfs_ul[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_ul))

i = -1
dfs_dl = []
for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_dl.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            data = os.path.join(datadir, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df = set_data(df)
            dfs_dl[i].append(df.copy())
            st.append(df['seq'].array[0])
            et.append(df['seq'].array[-1])
        st, et = max(st), min(et)
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            dfs_dl[i][j] = dfs_dl[i][j][(dfs_dl[i][j]['seq'] >= st) & (dfs_dl[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_dl))

/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm00/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm01/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm02/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm03/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm04/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm05/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm06/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm07/#01/data/udp_uplk_loss_latency.csv True
/Users/j

In [2]:
i = -1
dfs_ho = []
for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_ho.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            dbpath = os.path.join(datadir, exp, dev, trace, 'data')
            data = os.path.join(dbpath, [s for s in os.listdir(dbpath) if s.startswith('diag_log') and s.endswith('_rrc.csv')][0])
            # data = os.path.join(datadir, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df, _, _ = mi_parse_ho(df, tz=8)
            df['ho_type0'] = df['ho_type'].astype('string')
            df.loc[df['cause'].notna(), 'ho_type0'] = df['ho_type'].astype('string') + '_' + df['cause'].astype('string')
            df['ho_type0'] = df['ho_type0'].astype('category')
            dfs_ho[i].append(df.copy())
        #     st.append(df['seq'].array[0])
        #     et.append(df['seq'].array[-1])
        # st, et = max(st), min(et)
        # for j, (dev, schm) in enumerate(zip(devices, schemes)):
        #     dfs_ho[i][j] = dfs_ho[i][j][(dfs_ho[i][j]['seq'] >= st) & (dfs_ho[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_ho))

/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm00/#01/data/diag_log_sm00_2023-05-04_13-54-44_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm01/#01/data/diag_log_sm01_2023-05-04_13-54-44_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm02/#01/data/diag_log_sm02_2023-05-04_13-54-44_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm03/#01/data/diag_log_sm03_2023-05-04_13-54-44_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm04/#01/data/diag_log_sm04_2023-05-04_13-54-44_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm05/#01/data/diag_log_sm05_2023-05-04_13-54-44_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm06/#01/data/diag_log_sm06_2023-05-04_13-54-44_rrc.csv True
/Users/jackbedford/D

In [7]:
dfs_ho[0][1]

,start,end,ho_type,intr,sPCI,sFreq,tPCI,tFreq,snrPCI,tnrPCI,cause,others,st_scel,ePCI,ECI,eNB,BID,DL_Freq,DL_BW,UL_Freq,UL_BW,ePCI1,ECI1,eNB1,BID1,DL_Freq1,DL_BW1,UL_Freq1,UL_BW1,nrPCI,nrFreq,nrPCI1,nrFreq1,Timestamp,type_id,ho_type0
0,2023-05-04 13:55:12.181527,2023-05-04 13:55:12.197890,MN_HO,0.016363,178,1750,21,1750,178,<NA>,NaN,Set up 1 SCell.,1,178,<NA>,<NA>,<NA>,1750,NaN,<NA>,NaN,21,57265175,223692,3,1750,20 MHz,19750,20 MHz,178,631000,178,631000,2023-05-04 13:55:12.181527,RRC_OTA_Handover_Parsing,MN_HO
1,2023-05-04 13:55:14.522945,2023-05-04 13:55:14.538957,MN_HO,0.016012,21,1750,122,1750,178,<NA>,NaN,Set up 1 SCell.,1,21,57265175,223692,3,1750,20 MHz,19750,20 MHz,122,54303521,212123,3,1750,20 MHz,19750,20 MHz,178,631000,178,631000,2023-05-04 13:55:14.522945,RRC_OTA_Handover_Parsing,MN_HO
2,2023-05-04 13:55:20.968913,2023-05-04 13:55:20.985044,MN_HO,0.016131,122,1750,178,1750,178,<NA>,NaN,Set up 1 SCell.,1,122,54303521,212123,3,1750,20 MHz,19750,20 MHz,178,54562071,213133,3,1750,20 MHz,19750,20 MHz,178,631000,178,631000,2023-05-04 13:55:20.968913,RRC_OTA_Handover_Parsing,MN_HO
3,2023-05-04 13:55:30.463285,2023-05-04 13:55:30.480702,SN_HO,0.017417,178,1750,<NA>,<NA>,178,122,NaN,<NA>,0,178,54562071,213133,3,1750,20 MHz,19750,20 MHz,178,54562071,213133,3,1750,20 MHz,19750,20 MHz,178,631000,122,631000,2023-05-04 13:55:30.463285,RRC_OTA_Handover_Parsing,SN_HO
4,2023-05-04 13:55:36.335856,2023-05-04 13:55:36.351785,MN_HO,0.015929,178,1750,130,1750,122,<NA>,NaN,Set up 1 SCell.,1,178,54562071,213133,3,1750,20 MHz,19750,20 MHz,130,54303511,212123,3,1750,20 MHz,19750,20 MHz,122,631000,122,631000,2023-05-04 13:55:36.335856,RRC_OTA_Handover_Parsing,MN_HO
5,2023-05-04 13:55:37.578217,2023-05-04 13:55:37.594671,SN_HO,0.016454,130,1750,<NA>,<NA>,122,3,NaN,<NA>,0,130,54303511,212123,3,1750,20 MHz,19750,20 MHz,130,54303511,212123,3,1750,20 MHz,19750,20 MHz,122,631000,3,631000,2023-05-04 13:55:37.578217,RRC_OTA_Handover_Parsing,SN_HO
6,2023-05-04 13:55:40.520361,2023-05-04 13:55:40.537817,SN_HO,0.017456,130,1750,<NA>,<NA>,3,130,NaN,<NA>,0,130,54303511,212123,3,1750,20 MHz,19750,20 MHz,130,54303511,212123,3,1750,20 MHz,19750,20 MHz,3,631000,130,631000,2023-05-04 13:55:40.520361,RRC_OTA_Handover_Parsing,SN_HO
7,2023-05-04 13:55:44.360365,NaT,SCG_Failure,NaN,130,1750,<NA>,<NA>,130,<NA>,rlc-MaxNumRetx (2),<NA>,0,130,54303511,212123,3,1750,20 MHz,19750,20 MHz,130,54303511,212123,3,1750,20 MHz,19750,20 MHz,130,631000,130,631000,2023-05-04 13:55:44.360365,RRC_OTA_Handover_Parsing,SCG_Failure_rlc-MaxNumRetx (2)
8,2023-05-04 13:55:44.410497,2023-05-04 13:55:44.431846,SN_Rel,0.021349,130,1750,<NA>,<NA>,130,<NA>,NaN,Set up 1 SCell. Caused by scg-failure.,1,130,54303511,212123,3,1750,20 MHz,19750,20 MHz,130,54303511,212123,3,1750,20 MHz,19750,20 MHz,130,631000,<NA>,<NA>,2023-05-04 13:55:44.410497,RRC_OTA_Handover_Parsing,SN_Rel
9,2023-05-04 13:55:58.129607,2023-05-04 13:55:58.220817,MCG_Failure,0.091210,11,1400,130,<NA>,<NA>,<NA>,otherFailure (2),<NA>,0,11,54547468,213076,3,1400,10 MHz,19400,10 MHz,11,54547468,213076,3,1400,10 MHz,19400,10 MHz,<NA>,<NA>,<NA>,<NA>,2023-05-04 13:55:58.129607,RRC_OTA_Handover_Parsing,MCG_Failure_otherFailure (2)
